In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

# Custom labels for each experiment
custom_labels = {
    "seed_42": "Gaussian Noise",
    "seed_44": "Pink Noise (fmin=0.0, beta=1.0)",
    "seed_45": "Pink Noise (fmin=0.1, beta=1.0)",
    "seed_46": "Pink Noise (fmin=0.2, beta=1.0)",
    "seed_47": "Colored Noise (fmin=0.1, exponent=0.5)",
    "seed_48": "Colored Noise (fmin=0.1, exponent=1.5)",
}

rnd_labels = {
    "seed_49": "No RND",
    "seed_50": "RND (weight=0.5, lr=1e-4, hidden_dim=128)",
    "seed_51": "RND (weight=2.0, lr=1e-4, hidden_dim=128)",
    "seed_52": "RND (weight=1.0, lr=1e-3, hidden_dim=128)",
    "seed_53": "RND (weight=1.0, lr=1e-5, hidden_dim=128)",
    "seed_54": "RND (weight=1.0, lr=1e-4, hidden_dim=64)",
    "seed_55": "RND (weight=1.0, lr=1e-4, hidden_dim=256)",
    "seed_56": "RND (weight=1.0, lr=1e-4, hidden_dim=128)"
}

layernorm_labels = {
    "seed_49": "No LayerNorm",
    "seed_58": "LayerNorm (epsilon=1e-5)",
    "seed_59": "LayerNorm (epsilon=1e-4)",
    "seed_60": "LayerNorm (epsilon=1e-3)",
}

best_labels = {
    "seed_60": "LayerNorm (epsilon=1e-3)",
    "seed_44": "Pink Noise (fmin=0.0, beta=1.0)",
    "seed_50": "RND (weight=0.5, lr=1e-4, hidden_dim=128)",
    "seed_100": "LayerNorm + Pink Noise + RND",
    "seed_101": "RND + Pink Noise",
    "seed_102": "LayerNorm + RND",
    "seed_103": "Pink Noise + LayerNorm",
}



def get_winrates_from_file(file_path):
    try:
        data = np.load(file_path, allow_pickle=True).item()
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None
    return data.get("weak", None)

def load_all_winrates(results_dir):
    pattern = os.path.join(results_dir, "*", "*_winrates.npy")
    winrate_files = glob.glob(pattern)
    winrate_data = {}
    for file in winrate_files:
        winrates = get_winrates_from_file(file)
        if winrates is not None:
            label = os.path.basename(os.path.dirname(file))
            winrate_data[label] = winrates
    return winrate_data

def smooth_curve(data, window=3):
    if len(data) < window:
        return np.array(data)
    return np.convolve(data, np.ones(window)/window, mode='same')

def plot_winrates(winrate_data, label_dict, title, smoothing_window=3, output_file=None):
    plt.figure(figsize=(10, 7))

    lines = []
    labels = []
    
    for label, winrates in winrate_data.items():
        winrates_smoothed = smooth_curve(winrates, window=smoothing_window)
        x = np.arange(len(winrates_smoothed)) * 100
        label_to_plot = label_dict.get(label, label)

        line, = plt.plot(x, winrates_smoothed, label=label_to_plot, linewidth=2)
        lines.append(line)
        labels.append(label_to_plot)
    
    plt.xlim(0, 12000)
    plt.title(title, fontsize=20)
    plt.xlabel("Epochs", fontsize=18)
    plt.ylabel("Win Rate vs Weak Opponent", fontsize=18)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.grid(True, linestyle="--", alpha=0.6)

    plt.legend(lines, labels, fontsize=15, frameon=True, loc='best')
    
    if output_file:
        plt.savefig(output_file, bbox_inches='tight')
        print(f"Plot saved as {output_file}")
    else:
        plt.show()
    plt.close()

# Configuration
results_dir = "/home/ocal/Desktop/RL-Hockey/TD3/results_hockey/vs_weak"
base_output_dir = "/home/ocal/Desktop/RL-Hockey/TD3/plots/"

experiments = [
    {
        "name": "Noise",
        "labels": custom_labels,
        "title": "Training Performance with Different Noise Types",
        "output_file": os.path.join(base_output_dir, "td3_noise_experiment.png")
    },
    {
        "name": "RND",
        "labels": rnd_labels,
        "title": "Training Performance with Different RND Parameters",
        "output_file": os.path.join(base_output_dir, "td3_rnd_experiment.png")
    },
    {
        "name": "LayerNorm",
        "labels": layernorm_labels,
        "title": "Training Performance with Different Layernorm Parameters",
        "output_file": os.path.join(base_output_dir, "td3_layernorm_experiment.png")
    },
    {
        "name": "Best",
        "labels": best_labels,
        "title": "Training Performance with Combined vs. Individual Best Parameters",
        "output_file": os.path.join(base_output_dir, "td3_best_experiment.png")
    }
]

# Load data and generate plots
winrate_data = load_all_winrates(results_dir)
if winrate_data:
    for exp in experiments:
        filtered_data = {k: v for k, v in winrate_data.items() if k in exp["labels"]}
        if filtered_data:
            plot_winrates(
                filtered_data,
                exp["labels"],
                exp["title"],
                output_file=exp["output_file"]
            )
        else:
            print(f"No data found for {exp['name']} experiment.")
else:
    print("No winrate data found to plot.")

Plot saved as /home/ocal/Desktop/RL-Hockey/TD3/plots/td3_noise_experiment.png
Plot saved as /home/ocal/Desktop/RL-Hockey/TD3/plots/td3_rnd_experiment.png
Plot saved as /home/ocal/Desktop/RL-Hockey/TD3/plots/td3_layernorm_experiment.png
Plot saved as /home/ocal/Desktop/RL-Hockey/TD3/plots/td3_best_experiment.png
